In [2]:
# install biopython on Jupyter server.
import sys
!pip install biopython

In [1]:
import time
from Bio import Entrez

In [2]:
Entrez.email = "dbsnp-user@nih.gov" # provide your user email 
# RECOMMENDED: apply for API key from NCBI (https://ncbiinsights.ncbi.nlm.nih.gov/2017/11/02/new-api-keys-for-the-e-utilities/). 
# 10 queries per second with a valid API key, otherwise 3 queries per seconds are allowed for 'None'
Entrez.api_key = None

# entrez query (term) can be build and test online using web query builder (https://www.ncbi.nlm.nih.gov/snp/advanced) 
# esearch handle
eShandle = Entrez.esearch(db="snp",  # search dbSNP
                          #complex query for missense and pathogenic variants in LPL gene with global MAF betweeen 0 and 0.01.
                          term='LPL[All Fields] AND pathogenic[Clinical_Significance] AND missense variant[Function_Class] AND (00000.0000[GLOBAL_MAF] : 00000.0100[GLOBAL_MAF])', 
                          usehistory="y", #cache result on server for download in batches
                          retmax=20 # return 20 RSID max
                         )


In [3]:

# get esearch result
eSresult = Entrez.read(eShandle)

In [4]:
# review results 
for k in eSresult:
    print (k, ":", eSresult[k])
    
#Output: Web environment (&WebEnv) and query key (&query_key) parameters specifying the location on the Entrez history server of the list of UIDs matching the Entrez query
#https://www.ncbi.nlm.nih.gov/books/NBK25500/#chapter1.Storing_Search_Results
    

Count : 5
RetMax : 5
RetStart : 0
QueryKey : 1
WebEnv : NCID_1_3075896_130.14.18.97_9001_1567108675_833541832_0MetA0_S_MegaStore
IdList : ['386571803', '118204057', '52818902', '17850737', '268']
TranslationSet : []
TranslationStack : [{'Term': 'LPL[All Fields]', 'Field': 'All Fields', 'Count': '21533', 'Explode': 'N'}, {'Term': 'pathogenic[Clinical_Significance]', 'Field': 'Clinical_Significance', 'Count': '70373', 'Explode': 'N'}, 'AND', {'Term': 'missense variant[Function_Class]', 'Field': 'Function_Class', 'Count': '7741037', 'Explode': 'N'}, 'AND', {'Term': '00000.0000[GLOBAL_MAF]', 'Field': 'GLOBAL_MAF', 'Count': '0', 'Explode': 'N'}, {'Term': '00000.0100[GLOBAL_MAF]', 'Field': 'GLOBAL_MAF', 'Count': '0', 'Explode': 'N'}, 'RANGE', 'GROUP', 'AND']
QueryTranslation : LPL[All Fields] AND pathogenic[Clinical_Significance] AND missense variant[Function_Class] AND (00000.0000[GLOBAL_MAF] : 00000.0100[GLOBAL_MAF])


In [5]:
# get result RSIDs list 'Idlist'
# total rs count 
rslist = (eSresult['IdList'])

In [6]:
# retmax = 20 so print only 20 RSIDs
# additional results can be retrieved by batches
# download in batches example http://biopython.org/DIST/docs/tutorial/Tutorial.html#htoc139 or see below.
for rs in rslist:
    print(rs)

386571803
118204057
52818902
17850737
268


In [9]:
# get the WebEnv session cookie, and the QueryKey:

webenv = eSresult["WebEnv"]
query_key = eSresult["QueryKey"]
total_count = int(eSresult["Count"])
query_key = eSresult["QueryKey"]
retmax = 2 # return 2 rs per batch example

In [10]:
# sample codes adopted with modifications from http://biopython.org/DIST/docs/tutorial/Tutorial.html#htoc139.
fetch_count = 0
for start in range(0, total_count, retmax):
    end = min(total_count, start+retmax)
    print("Going to download record %i to %i" % (start+1, end))
    attempt = 0
    #fetch_count += 1
    while (attempt < 3):
        attempt += 1
        try:
            fetch_handle = Entrez.efetch(db="snp",
                                         #rettype="uilist", #available types [uilist | xml (use retmode=xml))
                                         retmode="xml",
                                         retstart=start,
                                         retmax=retmax,
                                         webenv=webenv,
                                         query_key=query_key )
        except HTTPError as err:
            if 500 <= err.code <= 599:
                print("Received error from server %s" % err)
                print("Attempt %i of 3" % attempt)
                time.sleep(15)
            else:
                raise
    if (fetch_handle):
        #print(fetch_handle)            
        data = fetch_handle.read()
        print(data)
        fetch_handle.close()



Going to download record 1 to 2
<?xml version="1.0" ?>
<ExchangeSet xmlns:xsi="https://www.w3.org/2001/XMLSchema-instance" xmlns="https://www.ncbi.nlm.nih.gov/SNP/docsum" xsi:schemaLocation="https://www.ncbi.nlm.nih.gov/SNP/docsum ftp://ftp.ncbi.nlm.nih.gov/snp/specs/docsum_eutils.xsd" ><DocumentSummary uid="386571803"><SNP_ID>268</SNP_ID><ALLELE_ORIGIN/><GLOBAL_MAFS><MAF><STUDY>1000Genomes</STUDY><FREQ>G=0.0052/26</FREQ></MAF><MAF><STUDY>ALSPAC</STUDY><FREQ>G=0.0187/72</FREQ></MAF><MAF><STUDY>Estonian</STUDY><FREQ>G=0.0239/107</FREQ></MAF><MAF><STUDY>ExAC</STUDY><FREQ>G=0.0134/1622</FREQ></MAF><MAF><STUDY>GnomAD</STUDY><FREQ>G=0.0145/455</FREQ></MAF><MAF><STUDY>GnomAD_exomes</STUDY><FREQ>G=0.0128/3212</FREQ></MAF><MAF><STUDY>NorthernSweden</STUDY><FREQ>G=0.0283/17</FREQ></MAF><MAF><STUDY>PAGE_STUDY</STUDY><FREQ>G=0.0050/391</FREQ></MAF><MAF><STUDY>TOPMED</STUDY><FREQ>G=0.0109/1370</FREQ></MAF><MAF><STUDY>TWINSUK</STUDY><FREQ>G=0.0200/74</FREQ></MAF></GLOBAL_MAFS><GLOBAL_POPULATION/><G